In [1]:
from get_data import get_data

get_data()

In [2]:
from pyspark.sql import SparkSession
from pathlib import Path

DATA_PATH = Path("/opt/workspace/data")

In [5]:
jar_postgres = Path.cwd() / "postgresql-42.5.1.jar"
master_url = "spark://spark-master:7077"
spark = SparkSession \
        .builder \
        .appName("FlightDelaysNotebook") \
        .config("spark.jars", str(jar_postgres)) \
        .master(master_url) \
        .getOrCreate()

22/11/27 16:25:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Lendo os dados baixados:

In [9]:
df = spark.read.csv(str(DATA_PATH), header=True)

In [10]:
df.columns

['FL_DATE',
 'OP_CARRIER',
 'OP_CARRIER_FL_NUM',
 'ORIGIN',
 'DEST',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'DEP_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'WHEELS_ON',
 'TAXI_IN',
 'CRS_ARR_TIME',
 'ARR_TIME',
 'ARR_DELAY',
 'CANCELLED',
 'CANCELLATION_CODE',
 'DIVERTED',
 'CRS_ELAPSED_TIME',
 'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'Unnamed: 27']

Escrevendo para um arquivo Parquet para comprimir e utiizar menos espaço no filesystem. Particionando por origem do voo para facilitar queries.

In [14]:
df \
    .drop("Unnamed: 27") \
    .write \
    .partitionBy("ORIGIN") \
    .parquet(str(DATA_PATH / "flight_delays") , mode="overwrite")

Deletando os csvs e relendo o DataFrame

In [15]:
for file in csv_files:
    file.unlink()

In [6]:
df = spark.read.parquet(str(DATA_PATH / "flight_delays"))

Escrevendo para a database Postgres (leva um tempo)

In [10]:
properties = {
    "user": "postgres",
    "password": "spark",
    "driver": "org.postgresql.Driver"
}
url = "jdbc:postgresql://demo-database:5432/postgres"
df \
    .write \
    .jdbc(url=url, table="flight_delays", mode="append", properties=properties)

In [38]:
import psycopg2 as psql

properties_psql = {
    "database": "postgres",
    "user": "postgres",
    "password": "spark",
    "host": "demo-database",
    "port": "5432"
}

conn = psql.connect(**properties_psql)
cur = conn.cursor()

In [39]:
cur.execute("select * from flight_delays limit 0")
cur.description

(Column(name='FL_DATE', type_code=25),
 Column(name='OP_CARRIER', type_code=25),
 Column(name='OP_CARRIER_FL_NUM', type_code=25),
 Column(name='DEST', type_code=25),
 Column(name='CRS_DEP_TIME', type_code=25),
 Column(name='DEP_TIME', type_code=25),
 Column(name='DEP_DELAY', type_code=25),
 Column(name='TAXI_OUT', type_code=25),
 Column(name='WHEELS_OFF', type_code=25),
 Column(name='WHEELS_ON', type_code=25),
 Column(name='TAXI_IN', type_code=25),
 Column(name='CRS_ARR_TIME', type_code=25),
 Column(name='ARR_TIME', type_code=25),
 Column(name='ARR_DELAY', type_code=25),
 Column(name='CANCELLED', type_code=25),
 Column(name='CANCELLATION_CODE', type_code=25),
 Column(name='DIVERTED', type_code=25),
 Column(name='CRS_ELAPSED_TIME', type_code=25),
 Column(name='ACTUAL_ELAPSED_TIME', type_code=25),
 Column(name='AIR_TIME', type_code=25),
 Column(name='DISTANCE', type_code=25),
 Column(name='CARRIER_DELAY', type_code=25),
 Column(name='WEATHER_DELAY', type_code=25),
 Column(name='NAS_DELAY

In [41]:
cur.execute('select "ORIGIN", count(*) from flight_delays group by "ORIGIN"')
cur.fetchall()

[('ABE', 30818),
 ('ABI', 20242),
 ('ABQ', 269861),
 ('ABR', 5072),
 ('ABY', 9669),
 ('ACK', 4797),
 ('ACT', 11691),
 ('ACV', 25292),
 ('ACY', 16902),
 ('ADK', 1040),
 ('ADQ', 6129),
 ('AEX', 32543),
 ('AGS', 31449),
 ('AKN', 709),
 ('ALB', 101076),
 ('ALO', 4128),
 ('AMA', 58619),
 ('ANC', 177173),
 ('APN', 3969),
 ('ART', 1595),
 ('ASE', 47755),
 ('ATL', 3903244),
 ('ATW', 37693),
 ('AUS', 468601),
 ('AVL', 45112),
 ('AVP', 19388),
 ('AZA', 5382),
 ('AZO', 21236),
 ('BDL', 226677),
 ('BET', 9767),
 ('BFF', 574),
 ('BFL', 31337),
 ('BGM', 5504),
 ('BGR', 12354),
 ('BHM', 164557),
 ('BIL', 35041),
 ('BIS', 30998),
 ('BJI', 5133),
 ('BKG', 6522),
 ('BLI', 7981),
 ('BLV', 1019),
 ('BMI', 31414),
 ('BNA', 564660),
 ('BOI', 137056),
 ('BOS', 1171494),
 ('BPT', 4701),
 ('BQK', 9659),
 ('BQN', 15706),
 ('BRD', 4537),
 ('BRO', 22686),
 ('BRW', 8575),
 ('BTM', 7164),
 ('BTR', 80276),
 ('BTV', 50895),
 ('BUF', 216565),
 ('BUR', 247125),
 ('BWI', 1003452),
 ('BZN', 42939),
 ('CAE', 67373),
 ('CA

In [43]:
df.groupBy("ORIGIN").count().collect()

[Row(ORIGIN='ATL', count=3903244),
 Row(ORIGIN='DFW', count=2546075),
 Row(ORIGIN='ORD', count=3001285),
 Row(ORIGIN='DEN', count=2300550),
 Row(ORIGIN='LAX', count=2133445),
 Row(ORIGIN='IAH', count=1672053),
 Row(ORIGIN='CLT', count=1334522),
 Row(ORIGIN='SFO', count=1612933),
 Row(ORIGIN='PHX', count=1720614),
 Row(ORIGIN='LAS', count=1472436),
 Row(ORIGIN='SEA', count=1142246),
 Row(ORIGIN='DTW', count=1290013),
 Row(ORIGIN='LGA', count=1080902),
 Row(ORIGIN='MSP', count=1258951),
 Row(ORIGIN='MCO', count=1227400),
 Row(ORIGIN='BOS', count=1171494),
 Row(ORIGIN='EWR', count=1182327),
 Row(ORIGIN='SLC', count=1146480),
 Row(ORIGIN='JFK', count=1058511),
 Row(ORIGIN='DCA', count=814016),
 Row(ORIGIN='BWI', count=1003452),
 Row(ORIGIN='PHL', count=835271),
 Row(ORIGIN='FLL', count=737310),
 Row(ORIGIN='MDW', count=870692),
 Row(ORIGIN='SAN', count=797453),
 Row(ORIGIN='IAD', count=605454),
 Row(ORIGIN='MIA', count=753511),
 Row(ORIGIN='TPA', count=686678),
 Row(ORIGIN='MEM', count=349